In [ ]:
# Package Declaration
# !pip install rasterio
!pip install netCDF4
# !pip install geopandas

# from osgeo import gdal
# from osgeo import ogr
# from osgeo import osr
# from osgeo import gdal_array
# from osgeo import gdalconst

import numpy as np
import pandas as pd
# import geopandas as gpd
import xarray as xr
import netCDF4 as nc

# import rasterio

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# CHIRPS 0.05 [DONE]

station_metadata_df = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/station_data/Final results_csv/station_list_final_v2.csv')
station_data_df = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_station_data_daily_v2.csv')
  
chirps_data_df = station_data_df[['Unnamed: 0']]

fname_16 = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CHIRPS/daily/05/chirps-v2.0.2016.days_p05.nc"
fname_17 = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CHIRPS/daily/05/chirps-v2.0.2017.days_p05.nc"
fname_18 = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CHIRPS/daily/05/chirps-v2.0.2018.days_p05.nc"
fname_19 = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CHIRPS/daily/05/chirps-v2.0.2019.days_p05.nc"
fname_20 = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CHIRPS/daily/05/chirps-v2.0.2020.days_p05.nc"

ds_16 = xr.open_dataset(fname_16)
ds_17 = xr.open_dataset(fname_17)
ds_18 = xr.open_dataset(fname_18)
ds_19 = xr.open_dataset(fname_19)
ds_20 = xr.open_dataset(fname_20)

for station in range(len(station_metadata_df.index)):
  print(station)

  station_id = station_metadata_df['STNID'].iloc[station]
  station_lat = station_metadata_df['LATITUDE'].iloc[station]
  station_lon = station_metadata_df['LONGITUDE'].iloc[station]

  clip_16 = ds_16.sel(latitude=station_lat, longitude=station_lon, method='nearest')
  clip_17 = ds_17.sel(latitude=station_lat, longitude=station_lon, method='nearest')
  clip_18 = ds_18.sel(latitude=station_lat, longitude=station_lon, method='nearest')
  clip_19 = ds_19.sel(latitude=station_lat, longitude=station_lon, method='nearest')
  clip_20 = ds_20.sel(latitude=station_lat, longitude=station_lon, method='nearest')
  
  val_16 = clip_16.precip.values
  val_17 = clip_17.precip.values
  val_18 = clip_18.precip.values
  val_19 = clip_19.precip.values
  val_20 = clip_20.precip.values
  
  satellite_data = np.concatenate((val_16,val_17))
  satellite_data = np.concatenate((satellite_data,val_18))
  satellite_data = np.concatenate((satellite_data,val_19))
  satellite_data = np.concatenate((satellite_data,val_20))

  chirps_data_df[station_id] = satellite_data

chirps_data_df = chirps_data_df.round(1)
chirps_data_df.to_csv("/content/gdrive/MyDrive/rainfall-forecast/verification_validation/chirps_0p05_station_daily.csv")

In [ ]:
# IMERG

station_metadata_df = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/station_data/Final results_csv/station_list_final_v2.csv')
station_data_df = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_station_data_daily_v2.csv')

station_data_df = station_data_df.set_index('Unnamed: 0')

imerg_data_df = pd.DataFrame().reindex_like(station_data_df)

date_range = pd.date_range('2016-01-01', '2020-12-31')
date_fname_list = date_range.strftime('%Y%m%d').to_list()
date_row_list = date_range.strftime('%Y-%m-%d').to_list()

# df_full_date = pd.DataFrame(date_list, columns=['YEARMODA'])
# df_full_date['YEARMODA'] = pd.to_datetime(df_full_date['YEARMODA'])
# df_full_date = df_full_date.set_index('YEARMODA')
# df_full_date['val'] = 0

for date in range(len(date_fname_list)):
  fname = '/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/IMERG/3B-DAY-L.MS.MRG.3IMERG.'+date_fname_list[date]+'-S000000-E235959.V06.nc4'
  print(fname)

  ds = xr.open_dataset(fname)

  for station in range(len(station_metadata_df.index)):
    print(station)

    station_id = station_metadata_df['STNID'].iloc[station]
    station_lat = station_metadata_df['LATITUDE'].iloc[station]
    station_lon = station_metadata_df['LONGITUDE'].iloc[station]

    clip = ds.sel(lat=station_lat, lon=station_lon, method='nearest')

    imerg_data_df[station_id].loc[date_row_list[date]] = clip.precipitationCal.values

imerg_data_df = imerg_data_df.round(1)
imerg_data_df.to_csv("/content/gdrive/MyDrive/rainfall-forecast/verification_validation/imerg_station_daily.csv")

In [ ]:
# CMORPH

station_metadata_df = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/station_data/Final results_csv/station_list_final_v2.csv')
station_data_df = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_station_data_daily_v2.csv')

station_data_df = station_data_df.set_index('Unnamed: 0')

cmorph_data_df = pd.DataFrame().reindex_like(station_data_df)

date_range = pd.date_range('2016-01-01', '2020-10-31')
date_fname_list = date_range.strftime('%Y%m%d').to_list()
date_row_list = date_range.strftime('%Y-%m-%d').to_list()

# df_full_date = pd.DataFrame(date_list, columns=['YEARMODA'])
# df_full_date['YEARMODA'] = pd.to_datetime(df_full_date['YEARMODA'])
# df_full_date = df_full_date.set_index('YEARMODA')
# df_full_date['val'] = 0

for date in range(len(date_fname_list)):
  fname = '/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH/daily_agg/daily_aggCMORPH_V1.0_ADJ_8km-30min_'+date_fname_list[date]+'.nc'

  ds = xr.open_dataset(fname)
  print(fname)

  lat_list = ds.Latitude.values
  lon_list = ds.Longitude.values

  precip_list = ds.precip.values

  for station in range(len(station_metadata_df.index)):
    print(date_fname_list[date], station)

    station_id = station_metadata_df['STNID'].iloc[station]
    station_lat = station_metadata_df['LATITUDE'].iloc[station]
    station_lon = station_metadata_df['LONGITUDE'].iloc[station]

    lat_idx = (np.abs(lat_list - station_lat)).argmin()
    lon_idx = (np.abs(lon_list - station_lon)).argmin()

    precip = precip_list[0,lat_idx,lon_idx]
    # precip_2 = precip_list[0,lon_idx,lat_idx]
    # print(lat_idx, lon_idx)
    # clip = ds.sel(lat=lat_idx, lon=lon_idx, method='nearest')

    cmorph_data_df[station_id].loc[date_row_list[date]] = precip

cmorph_data_df = cmorph_data_df.round(1)
cmorph_data_df.to_csv("/content/gdrive/MyDrive/rainfall-forecast/verification_validation/cmorph_dailyagg_station_daily.csv")

In [ ]:
# TAMSAT

station_metadata_df = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/station_data/Final results_csv/station_list_final_v2.csv')
station_data_df = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_station_data_daily_v2.csv')

# station_data_df = station_data_df[['Unnamed: 0']]
station_data_df = station_data_df.set_index('Unnamed: 0')

tamsat_data_df = pd.DataFrame().reindex_like(station_data_df)

fname = "/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/TAMSAT/bounding_box_daily/01-tamsatDaily.v3.1-1262304000-1622502000_28.417_41.194_-11.183_3.622.nc"

date_range = pd.date_range('2016-01-01', '2020-10-31')
date_fname_list = date_range.strftime('%Y%m%d').to_list()
date_row_list = date_range.strftime('%Y-%m-%d').to_list()

ds = xr.open_dataset(fname)

for station in range(len(station_metadata_df.index)):
  print(station)

  station_id = station_metadata_df['STNID'].iloc[station]
  station_lat = station_metadata_df['LATITUDE'].iloc[station]
  station_lon = station_metadata_df['LONGITUDE'].iloc[station]

  clip = ds.sel(lat=station_lat, lon=station_lon, method='nearest')
  clip = clip.sel(time=slice("2016-01-01", "2020-12-31"))
  
  val = clip.rfe.values
  # print(val)

  tamsat_data_df[station_id] = val

print(tamsat_data_df)

tamsat_data_df = tamsat_data_df.round(1)
tamsat_data_df.to_csv("/content/gdrive/MyDrive/rainfall-forecast/verification_validation/tamsat_rfe_station_daily.csv")